# 添数づけられた族，一般の直積

In [767]:
def series(func,N):
    return {n:func(n) for n in N}

In [768]:
def a1(n):
    assert n>0, 'n は自然数'
    return 1/n

def a2(n):
    return (-1)**(n-1)

添数集合 $\Lambda$ から集合 $A$ への写像
$$
a1: \Lambda \rightarrow A
$$

In [769]:
# 順序のある集合
N = range(1,1001)
sub_A1 = series(a1,N)
sub_A2 = series(a2,N)

In [770]:
n = 14  # 添数によって値を呼び出せる...
print(f'a1({n}) = {sub_A1[n]:.5f}')
print(f'a2({n}) = {sub_A2[n]:.0f}')

a1(14) = 0.07143
a2(14) = -1


In [771]:
# 順序のない集合
a3 = lambda x: x * 2
I = {a for a in 'nbg678iuyt5edfc'}
sub_A3 = series(a3,I)

i = 'n'
print(f'a1({i}) = {sub_A3[i]}')

a1(n) = nn


## 集合族

In [772]:
def power_set(S):  # 冪集合
    p_sets = [[]]
    for s in S:
        tmp = []
        for element in p_sets:
            tmp.append(element + [s])
        p_sets.extend(tmp)
    return [set(s) for s in p_sets]

def is_familiy_of_subset(As,X):
    return all([is_subset(A,X) for A in As])

In [773]:
X = {a for a in range(10)}
As = power_set(X)
I = range(len(As))

def a4(A):
    def func(i):
        return A[i]
    return func

A = series(a4(As),I)

In [774]:
is_familiy_of_subset(As,X)

True

### 集合族とその演算

In [780]:
def union(sets):
    sets = [{s} if isinstance(s,(int,str)) else s for s in sets]
    S = sets[0]
    for s in sets[1:]:
        S = {_ for _ in {*S, *s}}
    return S


def family_union(As):
    return union(As.values())


def intersection(sets):
    sets = [{s} if isinstance(s,(int,str)) else s for s in sets]
    S = sets[0]
    for s in sets[1:]:
        S = {_ for _ in S if _ in s}
    return S


def family_insec(As):
    return intersection(As.values())


def diff(A, S):
    return {_ for _ in S if _ not in A}


def get_values(As, keys):
    return {k:As[k] for k in keys}

In [781]:
print('和集合:', family_union(A))
print('共通部分:', family_insec(A))

和集合: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
共通部分: set()


In [782]:
print(diff(family_union(A),X))
print(family_insec({n:diff(A,X) for n,A in A.items()}))

set()
set()


In [783]:
J = [1,2,5,6,7,10]
sub_A = get_values(A,J)
print(sub_A)

print('補集合')
print(diff(family_union(sub_A),X))
print(family_insec({n:diff(A,X) for n,A in sub_A.items()}))

{1: {0}, 2: {1}, 5: {0, 2}, 6: {1, 2}, 7: {0, 1, 2}, 10: {1, 3}}
補集合
{4, 5, 6, 7, 8, 9}
{4, 5, 6, 7, 8, 9}


### 直積

In [784]:
def flatten(S):
    if not isinstance(S,list):
        return [S]
    x = []
    for s in S:
        x += flatten(s)
    return x

def direct_product(As):
    if isinstance(As,dict):
        As = list(As.values())
    X = []
    if len(As) == 1:
        return As[0]
    for a in As[0]:
        for b in direct_product(As[1:]):
            X += [flatten([a,b])]
    return X

In [785]:
J = [a for a in range(1,10)]
sub_A = get_values(A,J)
print(sub_A.values())

dict_values([{0}, {1}, {0, 1}, {2}, {0, 2}, {1, 2}, {0, 1, 2}, {3}, {0, 3}])


In [786]:
from functools import reduce
dpA = direct_product(sub_A)
print(len(dpA), len(reduce(lambda x,y: [0]*len(x)*len(y), sub_A.values())))

48 48


### 選出公理

$$
\forall \lambda \in \Lambda\ (A_{\lambda} \neq \varnothing) 
\implies \Pi_{\lambda \in \Lambda} A_{\lambda} \neq \varnothing
$$

と対偶にある

$$
\Pi_{\lambda \in \Lambda} A_{\lambda} = \varnothing
\implies \exists \lambda \in \Lambda\ (A_{\lambda} = \varnothing) 
$$

In [787]:
direct_product(A), A[0]

([], set())

### 射影

In [788]:
n = 40
l = 4

def projection(A,n,l):
    return A[n][l]

print(f'l={J[l]} の 点{dpA[n]} における座標:{projection(dpA,n,l)}')

l=5 の 点[0, 1, 1, 2, 2, 1, 2, 3, 0] における座標:2


### Th. 7

$f$ を $A$ から $B$ への写像とする．  
$$
(a)\  f が全射 \Leftrightarrow \exists I_{B}\ \left[ f \circ s = I_{B} となるような写像 I_{B}: B \rightarrow A \right]
$$

$$
(b)\  f が単射 \Leftrightarrow \exists I_{B}\ \left[ r \circ f = I_{A} となるような写像 I_{A}: B \rightarrow A \right]
$$

In [789]:
from collections import defaultdict
def is_surjective(A,B,func):  # 全射
    return all([b in func(A) for b in B])

def is_injective(A,B,func):  # 単射
    ans = defaultdict(list)
    inv_ans = defaultdict(list)
    for a in A:
        for b in func(a):
            ans[b].append(a)
            inv_ans[a].append(b)
    return all([len(v) == 1 for k,v in [*ans.items(),*inv_ans.items()]])

def is_bijective(A,B,func):  # 全単射
    return all([
        is_surjective(A,B,func),
        is_injective(A,B,func)
    ])

In [790]:
def relations(func, A):
    if isinstance(A,set):
        return {
            x: func(x) for x in A
        }
    elif isinstance(A,list):
        return {
            str(a): func(a) for a in A if func(a) is not None
        }

def image(relations):
    dmn = [a for a,b in relations.items() if isinstance(b,set)]
    img = [b for a,b in relations.items()]
    return union(dmn), union(img)

def is_subset(A,B):
    return all([x in B for x in A if x is not None])

def f(x):
    sets = 'abcdefg'
    if not isinstance(x,set):
        x = set([x])
    return {sets[a%len(sets)] for a in x}

def delta(relations):  # 逆対応
    inv_rlts = defaultdict(set)
    for key, vals in relations.items():
        for val in vals:
            inv_rlts[val].add(key)
    return inv_rlts

def inv_(func,P):
    rlt_P = relations(func,P)
    rlt = delta(rlt_P)
    def inv(x):
        if not isinstance(x,set):
            x = set([x])
        return union([rlt[a] for a in x])
    return inv

def is_map(A,func):  # 写像
    rlt = relations(func,A)
    return all([len(b) == 1 for a,b in rlt.items()])

(a)

In [791]:
A = {a for a in '1qaz2wsx3edc'}

def f(x=None, is_val=False):
    B = {0,1,2,3}
    if is_val:
        return B
    if not isinstance(x,(str,int)):
        return union([f(a) for a in x])
    if x in 'asdfghjkl;':
        return {0}
    if x in 'zxcvbnm,./':
        return {1}
    if x in 'qwertyuiop[]':
        return {2}
    else:
        return {3}
        

rlt_A = relations(f,A)
dmn_A, img_A = image(rlt_A)
B = f(is_val=True)
is_map(A,f)

True

In [792]:
is_surjective(A,B,f)

True

In [793]:
rlt_A

{'a': {0},
 'z': {1},
 '2': {3},
 'c': {1},
 's': {0},
 'x': {1},
 'd': {0},
 'w': {2},
 '1': {3},
 'e': {2},
 '3': {3},
 'q': {2}}

In [794]:
def s(x=None, is_val=False):
    A = {a for a in '1qaz2wsx3edc'}
    if is_val:
        return A
    if not isinstance(x,(str,int)):
        return union([s(a) for a in x])
    if x == 0:
        return {'a'}
    elif x == 1:
        return {'z'}
    elif x == 2:
        return {'w'}
    else:
        return {'3'}

rlt_B = relations(s,B)
dmn_B, img_B = image(rlt_B)
sub_A = s(is_val=True)
is_map(B,s)

True

In [795]:
def composit_map(F):
    def func(x):
        for f in F:
            x = f(x)
        return x
    return func

def is_identity(func,A):
    return all([{a}==func(a) for a in A])

In [796]:
i_b = composit_map([s,f])
rlt_i_b = relations(i_b,B)
print(rlt_i_b)
print('恒等写像か:', is_identity(i_b,B))

{0: {0}, 1: {1}, 2: {2}, 3: {3}}
恒等写像か: True


(b)

In [797]:
import random
A = {a for a in 'abcde'}

def f(x=None,is_val=False):
    B = {0,1,2,3,4,5,6,7,8,9,10}
    A = [a for a in 'abcde']
    X = {A[b]:b for b in B if b < len(A)}
    if is_val:
        return B
    if not isinstance(x,(str,int)):
        return union([f(a) for a in x])
    return {X[x]}

rlt_A = relations(f,A)
dmn_A, img_A = image(rlt_A)
B = f(is_val=True)
is_map(A,f)

True

In [798]:
is_injective(A,B,f)

True

In [799]:
rlt_A

{'a': {0}, 'c': {2}, 'd': {3}, 'b': {1}, 'e': {4}}

In [800]:
def r(x=None, is_val=False):
    A = [a for a in 'abcde']
    B = [b for b in range(11)]
    X = {b:A[b%len(A)] for b in B}
    if is_val:
        return set(A)
    if not isinstance(x,(str,int)):
        return union([r(a) for a in x])
    return {X[x]}

rlt_B = relations(r,B)
dmn_B, img_B = image(rlt_B)
A = r(is_val=True)
is_map(B,s)

True

In [801]:
i_a = composit_map([f,r])
rlt_i_a = relations(i_a,A)
print(rlt_i_a)
print('恒等写像か:', is_identity(i_a,A))

{'a': {'a'}, 'c': {'c'}, 'd': {'d'}, 'b': {'b'}, 'e': {'e'}}
恒等写像か: True


## 多変数写像

In [809]:
X = {0,1,2,3,4}
X_ps = power_set(X)  # 冪集合

A = X_ps[np.random.randint(len(X_ps))]
B = X_ps[np.random.randint(len(X_ps))]
print(f'A = {A}\nB = {B}')

A = {0, 1, 2, 4}
B = {2}


In [810]:
def phi(As):
    return union(As)

def psi(As):
    return intersection(As)

In [811]:
X_dp = direct_product([X_ps,X_ps])  # 冪集合同士の直積

rlt_phi = relations(phi,X_dp)

In [812]:
print('部分集合:', is_subset(img_phi,X))
print('要素:', img_phi in X_ps)

部分集合: True
要素: True


In [816]:
A = {(a,b,c) for a,b,c in zip(range(10),range(20,30),range(80,90))}

def f(a):
    return {reduce(lambda x,y: x+y, a)}

In [817]:
rlt_A = relations(f,A)
_, img_A = image(rlt_A)

In [818]:
rlt_A

{(7, 27, 87): {121},
 (6, 26, 86): {118},
 (2, 22, 82): {106},
 (8, 28, 88): {124},
 (5, 25, 85): {115},
 (9, 29, 89): {127},
 (0, 20, 80): {100},
 (1, 21, 81): {103},
 (4, 24, 84): {112},
 (3, 23, 83): {109}}